# import data

In [1]:
import pandas as pd
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

df_origin = pd.read_excel('stocks.xlsx')
df_contract_value =  df_origin.set_index('date')[['lastUpdateTime','Unnamed: 82']].reset_index()
df_contract_value.columns = ['code','value','industry']
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_wind =  w.wss(f"{stk_code_str}", "sec_name", "tradeDate=20221102;",usedf=True)[1]
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_info = df_contract_value.set_index('code').loc[stk_code_list]
# pd.set_option('display.max_rows', None)
df_stk_info['name'] = df_stk_wind['SEC_NAME']
df_stk_clean = df_stk_info[df_stk_info['industry']!=0].fillna('工业').sort_values(by='industry').reset_index()
df_stk_clean = df_stk_clean[['code','name','value','industry']]
# pd.set_option('display.max_rows', None)
df_stk_clean['value'] = df_stk_clean['value'].apply(lambda x: format(x, 'f'))

22.9.1.52437
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
218
218


In [2]:
df_stk_clean
idsty_stk_dic = {}
for index,row in df_stk_clean.iterrows():
    tmp_code = row['code']
    tmp_industry = row['industry']
    if tmp_industry not in idsty_stk_dic:
        idsty_stk_dic[tmp_industry] = [tmp_code]
    else:
        idsty_stk_dic[tmp_industry].append(tmp_code)
print(idsty_stk_dic.keys())
for k,v in idsty_stk_dic.items():
    print(k,v)  

dict_keys(['交通运输', '传媒', '公用事业', '农林牧渔', '医药生物', '商贸零售', '国防军工', '基础化工', '家用电器', '工业', '建筑材料', '建筑装饰', '房地产', '有色金属', '机械设备', '汽车', '煤炭', '环保', '电力设备', '电子', '石油石化', '社会服务', '美容护理', '计算机', '轻工制造', '通信', '钢铁', '银行', '非银金融', '食品饮料'])
交通运输 ['601006.SH', '000088.SZ', '600798.SH', '000099.SZ']
传媒 ['002027.SZ', '002291.SZ', '002555.SZ', '002624.SZ', '300251.SZ', '300315.SZ', '300518.SZ']
公用事业 ['601985.SH', '600452.SH', '601222.SH', '600163.SH', '600021.SH']
农林牧渔 ['002556.SZ', '002714.SZ', '601952.SH', '002505.SZ']
医药生物 ['300601.SZ', '300003.SZ', '600645.SH', '600436.SH', '600276.SH', '600267.SH', '300122.SZ', '600993.SH', '600055.SH', '300244.SZ', '300630.SZ', '000661.SZ', '002252.SZ', '002044.SZ', '688658.SH', '300760.SZ', '002675.SZ', '600763.SH', '603259.SH', '000931.SZ', '300233.SZ']
商贸零售 ['600859.SH', '600729.SH', '601888.SH', '000861.SZ', '600738.SH', '600790.SH']
国防军工 ['600316.SH', '000738.SZ', '600150.SH', '002013.SZ', '000519.SZ', '002625.SZ', '600372.SH']
基础化工 ['000792.SZ', '600989

In [6]:
# cbond_list = ['113516.SH','110043.SH',
# ]
cbond_list = idsty_stk_dic['交通运输']  
# ###############################################################
before_trade_date = '20211213'#监测当日前一个交易日
present_date = '20211214' #监测当日
# next_trade_date = '20220917' #监测当日下一个自然日
# ###############################################################

start_date = pd.to_datetime(before_trade_date) 

end_date = pd.to_datetime('20220729') #多一个月没关系，切片的时候，不需要这多出来的一个月的对子会切掉

cbond_price = pd.DataFrame([])
for code in cbond_list:
    price_df = w.wsi(code, "close", start_date, end_date,usedf=True)[1]   #只要了收盘价
    price_df.columns = [code]
    cbond_price = pd.concat([cbond_price,price_df],axis=1)
cbond_price = cbond_price.ffill()    
# cbond_price = cbond_price[cbond_price.index.time!=time(15,0,0)]
# cbond_price = cbond_price.loc['2022-09-09 15:00:00':] #从前一个交易日的收盘价开始截取
cbond_price_begin_time = before_trade_date[:4]+'-' + \
                    before_trade_date[4:6]+'-' + \
                    before_trade_date[6:8]+' 15:00:00'
cbond_price = cbond_price.loc[cbond_price_begin_time:]
cbond_price

,601006.SH,000088.SZ,600798.SH,000099.SZ
2021-12-13 15:00:00,6.36,5.29,4.17,8.10
2021-12-14 09:30:00,6.35,5.28,4.14,8.10
2021-12-14 09:31:00,6.35,5.27,4.14,8.12
2021-12-14 09:32:00,6.35,5.28,4.14,8.12
2021-12-14 09:33:00,6.33,5.27,4.14,8.13
...,...,...,...,...
2022-07-28 14:56:00,6.04,5.06,4.02,8.32
2022-07-28 14:57:00,6.04,5.06,4.02,8.32
2022-07-28 14:58:00,6.04,5.06,4.02,8.32
2022-07-28 14:59:00,6.04,5.06,4.02,8.32


# return calculate

In [7]:
cbond_return = pd.DataFrame([])
# tmp_stk_code = '601006.SH'
# cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_price.iloc[0,:].tolist() # 初始时刻的stock前收盘价

for index,row in cbond_price.iterrows():
    
    # cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    for i in range(len(cbond_list)):
        cbond_return.loc[index,cbond_list[i]] = row[cbond_list[i]]/stock_preclose[i] - 1
        if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
            # cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
            stock_preclose[i] = row[cbond_list[i]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [8]:
cbond_price

,601006.SH,000088.SZ,600798.SH,000099.SZ
2021-12-13 15:00:00,6.36,5.29,4.17,8.10
2021-12-14 09:30:00,6.35,5.28,4.14,8.10
2021-12-14 09:31:00,6.35,5.27,4.14,8.12
2021-12-14 09:32:00,6.35,5.28,4.14,8.12
2021-12-14 09:33:00,6.33,5.27,4.14,8.13
...,...,...,...,...
2022-07-28 14:56:00,6.04,5.06,4.02,8.32
2022-07-28 14:57:00,6.04,5.06,4.02,8.32
2022-07-28 14:58:00,6.04,5.06,4.02,8.32
2022-07-28 14:59:00,6.04,5.06,4.02,8.32


In [9]:
import numpy as np
cbond_return

,601006.SH,000088.SZ,600798.SH,000099.SZ
2021-12-13 15:00:00,0.000000,0.000000,0.000000,0.000000
2021-12-14 09:30:00,-0.001572,-0.001890,-0.007194,0.000000
2021-12-14 09:31:00,-0.001572,-0.003781,-0.007194,0.002469
2021-12-14 09:32:00,-0.001572,-0.001890,-0.007194,0.002469
2021-12-14 09:33:00,-0.004717,-0.003781,-0.007194,0.003704
...,...,...,...,...
2022-07-28 14:56:00,0.000000,-0.001972,-0.004950,0.024631
2022-07-28 14:57:00,0.000000,-0.001972,-0.004950,0.024631
2022-07-28 14:58:00,0.000000,-0.001972,-0.004950,0.024631
2022-07-28 14:59:00,0.000000,-0.001972,-0.004950,0.024631


In [10]:
cbond_return.corr(method='pearson')

,601006.SH,000088.SZ,600798.SH,000099.SZ
601006.SH,1.000000,0.303435,0.269730,0.159971
000088.SZ,0.303435,1.000000,0.639468,0.416660
600798.SH,0.269730,0.639468,1.000000,0.395649
000099.SZ,0.159971,0.416660,0.395649,1.000000


In [11]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)
y = cbond_return['600798.SH'].tolist()
x = cbond_return['000088.SZ'].tolist()
cointegration_test(y, x)

(-11.379033387849093,
 8.602053066620114e-21,
 42,
 36500,
 {'1%': -3.43052917150548,
  '5%': -2.8616191894802756,
  '10%': -2.5668121500536687},
 -309664.2514073367)